In [ ]:
!pip install gender-guesser

     |████████████████████████████████| 379 kB 5.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import math
from scipy.special import rel_entr
import gender_guesser.detector as gender


In [ ]:
df=pd.read_csv('CUYAHOGA.txt')
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (25,58,64,66,67,74,82,87,93,96,97,103) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SOS_VOTERID,COUNTY_NUMBER,COUNTY_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,...,GENERAL-11/06/2018,PRIMARY-05/07/2019,PRIMARY-09/10/2019,GENERAL-11/05/2019,PRIMARY-03/17/2020,GENERAL-11/03/2020,PRIMARY-05/04/2021,PRIMARY-08/03/2021,PRIMARY-09/14/2021,GENERAL-11/02/2021
0,OH0015867360,18,2120121,BRESLIN,THOMAS,J,III,1986-06-05,2019-08-27,ACTIVE,...,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN
1,OH0020007076,18,2362083,KASTANIS,AARON,L,NaN,1987-08-30,2019-12-12,ACTIVE,...,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN
2,OH0025485218,18,2921937,BODAIR,RAYIK,SALEM,NaN,1956-05-18,2020-02-24,ACTIVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OH0025727101,18,2945437,LOZIER,MELANIE,W,NaN,1990-09-18,2020-08-21,ACTIVE,...,X,NaN,NaN,NaN,NaN,X,NaN,D,X,X
4,OH0025727957,18,2946208,BURR,DAVID,DANIEL,NaN,1968-12-31,2020-08-24,ACTIVE,...,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872233,OH0018843818,18,2216108,TODD,MICHAEL,NaN,NaN,1959-03-15,2014-06-16,ACTIVE,...,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN
872234,OH0015299177,18,1953728,MLAKAR,TRAVIS,M,NaN,1975-01-05,2014-07-11,ACTIVE,...,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X
872235,OH0014970786,18,991609,KING,MATTHEW,C,NaN,1976-04-08,2015-07-02,ACTIVE,...,X,NaN,NaN,NaN,D,X,NaN,NaN,X,X
872236,OH0023314127,18,2682520,PLESEC,JONAH,JAQUISE,NaN,1997-01-12,2015-08-28,ACTIVE,...,X,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,X


In [ ]:
# Convert date to datatime type
df['DATE_OF_BIRTH'] =  pd.to_datetime(df['DATE_OF_BIRTH'])
df['REGISTRATION_DATE'] = pd.to_datetime(df['REGISTRATION_DATE'])

In [ ]:
# calculate age and registration_age: when did this person registered

current_year = datetime.datetime.now().year #get current year

df['Age'] = current_year - df["DATE_OF_BIRTH"].dt.year
df['Registration_age'] = df["REGISTRATION_DATE"].dt.year - df["DATE_OF_BIRTH"].dt.year

In [ ]:
# Predict gender from first name

d = gender.Detector(case_sensitive=False)
# df['Gender'] = df.FIRST_NAME.apply(lambda x: d.get_gender(u'%s'%x, u'usa')) # with 'usa'
df['Gender_1'] = df.FIRST_NAME.apply(lambda x: d.get_gender(u'%s'%x)) # without 'usa'
df['Gender_2'] = df.MIDDLE_NAME.apply(lambda x: d.get_gender(u'%s'%x))

In [ ]:
# Select only necessary columns

df = df.loc[:, ["SOS_VOTERID", "FIRST_NAME", "MIDDLE_NAME", "SUFFIX", "LAST_NAME", "DATE_OF_BIRTH", "RESIDENTIAL_ADDRESS1", "RESIDENTIAL_CITY", "RESIDENTIAL_ZIP", "Age", "Registration_age", "Gender_1", "Gender_2"]]

In [ ]:
df

,SOS_VOTERID,FIRST_NAME,MIDDLE_NAME,SUFFIX,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender_1,Gender_2
0,OH0015867360,THOMAS,J,III,BRESLIN,1986-06-05,11510 GLEN VALLEY DR,BRECKSVILLE,44141,36,33,male,unknown
1,OH0020007076,AARON,L,NaN,KASTANIS,1987-08-30,15992 WEDGEWOOD LN,STRONGSVILLE,44149,35,32,male,unknown
2,OH0025485218,RAYIK,SALEM,NaN,BODAIR,1956-05-18,1640 WINTON AVE,LAKEWOOD,44107,66,64,unknown,male
3,OH0025727101,MELANIE,W,NaN,LOZIER,1990-09-18,1468 W 48TH ST,CLEVELAND,44102,32,30,female,unknown
4,OH0025727957,DAVID,DANIEL,NaN,BURR,1968-12-31,24215 BRUCE RD,BAY VILLAGE,44140,54,52,male,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...
872233,OH0018843818,MICHAEL,NaN,NaN,TODD,1959-03-15,1530 COVENTRY RD,EAST CLEVELAND,44118,63,55,male,mostly_female
872234,OH0015299177,TRAVIS,M,NaN,MLAKAR,1975-01-05,54 WALNUT ST,CHAGRIN FALLS,44022,47,39,male,unknown
872235,OH0014970786,MATTHEW,C,NaN,KING,1976-04-08,4291 W 180TH ST,CLEVELAND,44135,46,39,male,unknown
872236,OH0023314127,JONAH,JAQUISE,NaN,PLESEC,1997-01-12,19603 MUSKOKA AVE,CLEVELAND,44119,25,18,male,unknown


In [ ]:
# df.groupby(['Gender_1']).size()

In [ ]:
# df.groupby(['Gender_1', 'Gender_2']).size()

In [ ]:
def update_gender(x):
  # if not clearly identified as male or female
  if x[-2] != 'male' and x[-2] != 'female':

    # if has a suffix then male
    if pd.notnull(x.SUFFIX):
      return 'male'
    
    # if mostly_male/female by first name, then male/female
    elif x[-2] == 'mostly_male':
      return 'male'
    elif x[-2] == 'mostly_female':
      return 'female'
    
    # if 'andy' or 'unknown' by first name, then by middle name if it is not unknown
    # elif x[-2] == 'andy' and x[-1] != 'unknown':
    #   return x[-1]
    elif x[-2] == 'unknown' and x[-1] != 'unknown':
      if x[-1] == 'mostly_male':
        return 'male'
      elif x[-1] == 'mostly_female':
        return 'female'
      else:
        return x[-1]

  else: 
    return x[-2]

In [ ]:
# update unclear genders

df['Gender'] = df.apply(lambda x: update_gender(x), axis = 1)

In [ ]:
df.groupby('Gender').size()

Gender
andy         334
female    421918
male      388788
dtype: int64

In [ ]:
population = df.loc[:, ["SOS_VOTERID", "FIRST_NAME", "LAST_NAME", "DATE_OF_BIRTH", "RESIDENTIAL_ADDRESS1", "RESIDENTIAL_CITY", "RESIDENTIAL_ZIP", "Age", "Registration_age", "Gender"]]

In [ ]:
population

,SOS_VOTERID,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender
0,OH0015867360,THOMAS,BRESLIN,1986-06-05,11510 GLEN VALLEY DR,BRECKSVILLE,44141,36,33,male
1,OH0020007076,AARON,KASTANIS,1987-08-30,15992 WEDGEWOOD LN,STRONGSVILLE,44149,35,32,male
2,OH0025485218,RAYIK,BODAIR,1956-05-18,1640 WINTON AVE,LAKEWOOD,44107,66,64,male
3,OH0025727101,MELANIE,LOZIER,1990-09-18,1468 W 48TH ST,CLEVELAND,44102,32,30,female
4,OH0025727957,DAVID,BURR,1968-12-31,24215 BRUCE RD,BAY VILLAGE,44140,54,52,male
...,...,...,...,...,...,...,...,...,...,...
872233,OH0018843818,MICHAEL,TODD,1959-03-15,1530 COVENTRY RD,EAST CLEVELAND,44118,63,55,male
872234,OH0015299177,TRAVIS,MLAKAR,1975-01-05,54 WALNUT ST,CHAGRIN FALLS,44022,47,39,male
872235,OH0014970786,MATTHEW,KING,1976-04-08,4291 W 180TH ST,CLEVELAND,44135,46,39,male
872236,OH0023314127,JONAH,PLESEC,1997-01-12,19603 MUSKOKA AVE,CLEVELAND,44119,25,18,male


### 1. Surname Filter

In [ ]:
m = population.groupby(['LAST_NAME'])['LAST_NAME'].transform('size')

In [ ]:
surname_filter = population.loc[m >=3]
surname_filter

,SOS_VOTERID,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender
0,OH0015867360,THOMAS,BRESLIN,1986-06-05,11510 GLEN VALLEY DR,BRECKSVILLE,44141,36,33,male
1,OH0020007076,AARON,KASTANIS,1987-08-30,15992 WEDGEWOOD LN,STRONGSVILLE,44149,35,32,male
2,OH0025485218,RAYIK,BODAIR,1956-05-18,1640 WINTON AVE,LAKEWOOD,44107,66,64,male
3,OH0025727101,MELANIE,LOZIER,1990-09-18,1468 W 48TH ST,CLEVELAND,44102,32,30,female
4,OH0025727957,DAVID,BURR,1968-12-31,24215 BRUCE RD,BAY VILLAGE,44140,54,52,male
...,...,...,...,...,...,...,...,...,...,...
872233,OH0018843818,MICHAEL,TODD,1959-03-15,1530 COVENTRY RD,EAST CLEVELAND,44118,63,55,male
872234,OH0015299177,TRAVIS,MLAKAR,1975-01-05,54 WALNUT ST,CHAGRIN FALLS,44022,47,39,male
872235,OH0014970786,MATTHEW,KING,1976-04-08,4291 W 180TH ST,CLEVELAND,44135,46,39,male
872236,OH0023314127,JONAH,PLESEC,1997-01-12,19603 MUSKOKA AVE,CLEVELAND,44119,25,18,male


### 2. Age Filter

In [ ]:
age_filter = population[(population.Age >= 18) & (population.Age <=65)]
age_filter

,SOS_VOTERID,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender
0,OH0015867360,THOMAS,BRESLIN,1986-06-05,11510 GLEN VALLEY DR,BRECKSVILLE,44141,36,33,male
1,OH0020007076,AARON,KASTANIS,1987-08-30,15992 WEDGEWOOD LN,STRONGSVILLE,44149,35,32,male
3,OH0025727101,MELANIE,LOZIER,1990-09-18,1468 W 48TH ST,CLEVELAND,44102,32,30,female
4,OH0025727957,DAVID,BURR,1968-12-31,24215 BRUCE RD,BAY VILLAGE,44140,54,52,male
5,OH0025977035,SHONTESE,WASHINGTON,1995-10-31,24100 S WOODLAND RD,SHAKER HTS,44122,27,25,None
...,...,...,...,...,...,...,...,...,...,...
872232,OH0014529466,LINDA,BRYANT,1958-05-31,12905 CARPENTER RD,GARFIELD HTS,44125,64,55,female
872233,OH0018843818,MICHAEL,TODD,1959-03-15,1530 COVENTRY RD,EAST CLEVELAND,44118,63,55,male
872234,OH0015299177,TRAVIS,MLAKAR,1975-01-05,54 WALNUT ST,CHAGRIN FALLS,44022,47,39,male
872235,OH0014970786,MATTHEW,KING,1976-04-08,4291 W 180TH ST,CLEVELAND,44135,46,39,male


In [ ]:
q = age_filter.groupby("Age").size().reset_index()
q.columns=["Age", "counts"]
q_age = q.Age.to_numpy()
q_dist = (q.counts/q.shape[0]).to_numpy()
# normalization
q_dist = q_dist/sum(q_dist)

### 3. Registration Filter

In [ ]:
registration_filter = age_filter[(age_filter.Registration_age <= 25) & (age_filter.Registration_age >= 0)]
registration_filter

,SOS_VOTERID,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender
5,OH0025977035,SHONTESE,WASHINGTON,1995-10-31,24100 S WOODLAND RD,SHAKER HTS,44122,27,25,None
6,OH0026064371,CHAD,KEGELMAYA,2001-08-26,4545 COLLEGE RD,SOUTH EUCLID,44121,21,19,male
21,OH0015371997,DALE,ANDRERSON,1984-06-05,3329 STORER AVE,CLEVELAND,44109,38,22,male
23,OH0018799915,DANIEL,RIEBEL,1985-04-15,29107 EDGEWOOD RD,BAY VILLAGE,44140,37,21,male
24,OH0021281369,BRYCE,MIKOL,1992-12-09,8160 DAVENTREE DR,BRECKSVILLE,44141,30,18,male
...,...,...,...,...,...,...,...,...,...,...
872204,OH0024988213,ARION,GREGORY,1999-01-06,5244 E 113TH ST,GARFIELD HTS,44125,23,20,male
872207,OH0025455914,NATHAN,BOOTS,2002-04-26,6708 CHARLES AVE,PARMA,44129,20,18,male
872229,OH0019635885,AMANDA,FREDERICK,1989-12-29,6822 PELHAM DR,PARMA,44129,33,24,female
872231,OH0022291085,REBECCA,HANNA,1991-05-05,14032 LAKEWOOD HTS BLVD,LAKEWOOD,44107,31,23,female


### Merge registration_filter & surname


In [ ]:
prep_pop = pd.merge(registration_filter, surname_filter, how='inner', on=['SOS_VOTERID', 'FIRST_NAME', 'LAST_NAME', 'DATE_OF_BIRTH', 'RESIDENTIAL_ADDRESS1', 'RESIDENTIAL_CITY', 'RESIDENTIAL_ZIP', 'Age', 'Registration_age', 'Gender'])
prep_pop = prep_pop[prep_pop.Gender.isin(['male', 'female'])]
prep_pop

,SOS_VOTERID,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender
1,OH0018799915,DANIEL,RIEBEL,1985-04-15,29107 EDGEWOOD RD,BAY VILLAGE,44140,37,21,male
2,OH0021281369,BRYCE,MIKOL,1992-12-09,8160 DAVENTREE DR,BRECKSVILLE,44141,30,18,male
3,OH0022465016,TIMOTHY,GAVIN,1988-07-04,4016 TORRINGTON AVE,PARMA,44134,34,25,male
4,OH0023541454,ABIGAIL,NORMAN,1998-06-02,1533 CLARENCE AVE,LAKEWOOD,44107,24,18,female
5,OH0022009256,ASHLEY,ROZIER,1991-12-05,1737 CATALPA RD,CLEVELAND,44112,31,25,female
...,...,...,...,...,...,...,...,...,...,...
149225,OH0024988213,ARION,GREGORY,1999-01-06,5244 E 113TH ST,GARFIELD HTS,44125,23,20,male
149226,OH0025455914,NATHAN,BOOTS,2002-04-26,6708 CHARLES AVE,PARMA,44129,20,18,male
149227,OH0019635885,AMANDA,FREDERICK,1989-12-29,6822 PELHAM DR,PARMA,44129,33,24,female
149228,OH0022291085,REBECCA,HANNA,1991-05-05,14032 LAKEWOOD HTS BLVD,LAKEWOOD,44107,31,23,female


### Drop duplicates keeping the first one

In [ ]:
def kl_cal(sample, q_age, q_dist):
  p = sample.groupby("Age").size().reset_index()
  p.columns=["Age", "counts"]
  p_age = p.Age.to_numpy()
  p_dist = (p.counts/p.shape[0]).to_numpy()
  p_dist = p_dist/sum(p_dist)

  kl_val = sum(rel_entr(p_dist, q_dist))

  return kl_val

In [ ]:
# obtain the age distribution from age_filter, age_filter is the sliced version from the original dataset on certain ages, from 18 to 81
# pop_gender_dist = [1-0.52043280794, 0.52043280794] #calculated it manually :/
counts = age_filter.groupby('Age').size().to_numpy() 
weights = counts / sum(counts)

prep_pop_copy = prep_pop
age_group = np.round(weights*1000)
age_group[5] = 18
age_group[19] = 22
age_group[23] = 20
age_group[45] = 23
age_group[46] = 23

sample = pd.DataFrame()
for i in range(18, 66):
     surname_group = []
     while(len(surname_group)!=age_group[i-18]):
         sample_from_group = prep_pop_copy[prep_pop_copy.Age == i].sample(int(age_group[i-18]))
         surname_group = sample_from_group.groupby('LAST_NAME').size().reset_index()
     sample = pd.concat([sample, sample_from_group])
     prep_pop_copy = prep_pop_copy[~prep_pop_copy.LAST_NAME.isin(surname_group.LAST_NAME)]
kl_val = kl_cal(sample, q_age, q_dist)

sample.groupby('Gender').size().reset_index()

,Gender,0
0,female,520
1,male,480


In [ ]:
sample_full = df[df.SOS_VOTERID.isin(sample.SOS_VOTERID)]
sample_full

,SOS_VOTERID,FIRST_NAME,MIDDLE_NAME,SUFFIX,LAST_NAME,DATE_OF_BIRTH,RESIDENTIAL_ADDRESS1,RESIDENTIAL_CITY,RESIDENTIAL_ZIP,Age,Registration_age,Gender_1,Gender_2,Gender
69,OH0019969581,EMILY,H,NaN,EGAN,1990-06-12,270 CORNWALL RD,ROCKY RIVER,44116,32,18,female,unknown,female
1579,OH0015234587,KATHERINE,A,NaN,BETTERS,1980-09-14,3950 ROCKY RIVER DR,CLEVELAND,44111,42,25,female,unknown,female
4109,OH0015227905,JOHNNY,J,JR,PERKINS,1983-08-05,1436 IROQUOIS AVE,MAYFIELD HTS,44124,39,25,male,unknown,male
5102,OH0015195091,JEFFERY,M,NaN,ENGRAM,1979-02-15,3034 ALBION RD,CLEVELAND,44120,43,21,male,unknown,male
5779,OH0024263172,SEAN,PATRICK,NaN,GORMAN,1992-01-08,2152 CHESTERLAND AVE,LAKEWOOD,44107,30,25,male,male,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869123,OH0014565353,JENNIFER,D,NaN,MCDANIEL,1961-11-26,4150 STONEHAVEN RD,SOUTH EUCLID,44121,61,22,female,unknown,female
869719,OH0022707178,JOSEPH,PATRICK HARRY,NaN,CHRISTENSEN,1990-02-04,2676 MAYFIELD RD,CLEVELAND HTS,44106,32,24,male,unknown,male
870169,OH0024984693,ANGELA,MARIE,NaN,ODONNELL,1994-10-07,8275 NEWCOMB DR,PARMA,44129,28,25,female,female,female
870742,OH0014446794,LESLIE,J,NaN,TRAYLOR,1957-03-07,30919 ARLINGTON CIR,BAY VILLAGE,44140,65,23,mostly_female,unknown,female


In [ ]:
sample_full.to_csv('sample_data.csv', sep='\t', index=False, encoding='utf-8')